In [1]:
import os
import sys

os.environ["JAVA_HOME"] = "JDK 8"
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("assessment").getOrCreate()


25/02/14 18:27:44 WARN Utils: Your hostname, Mint-T470 resolves to a loopback address: 127.0.1.1; using 192.168.1.249 instead (on interface wlp4s0)
25/02/14 18:27:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/14 18:27:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Exercise 4

In [3]:
df_csv = spark.read.csv("HR-Employee-Attrition.csv", header=True, inferSchema=True)
df_csv.show(5)

25/02/14 18:28:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+---------+-----------------+---------+--------------------+----------------+---------+--------------+-------------+--------------+-----------------------+------+----------+--------------+--------+--------------------+---------------+-------------+-------------+-----------+------------------+------+--------+-----------------+-----------------+------------------------+-------------+----------------+-----------------+---------------------+---------------+--------------+------------------+-----------------------+--------------------+
|Age|Attrition|   BusinessTravel|DailyRate|          Department|DistanceFromHome|Education|EducationField|EmployeeCount|EmployeeNumber|EnvironmentSatisfaction|Gender|HourlyRate|JobInvolvement|JobLevel|             JobRole|JobSatisfaction|MaritalStatus|MonthlyIncome|MonthlyRate|NumCompaniesWorked|Over18|OverTime|PercentSalaryHike|PerformanceRating|RelationshipSatisfaction|StandardHours|StockOptionLevel|TotalWorkingYears|TrainingTimesLastYear|WorkLifeBalanc

Exercise 6

In [4]:
df_dropped = df_csv.drop("EmployeeCount", "Over18", "StandardHours")
#EmployeeCount has the same value (1) for each entry
#All employees are 18+
#StandardHours are all the same value (80)

In [5]:
import re

# new_columns = {column: re.sub( r"([a-z])([A-Z])", r"\1_\2", column).lower() for column in df_dropped.columns}
new_columns = [re.sub(r"([a-z])([A-Z])", r"\1_\2", column).lower() for column in df_dropped.columns]
df_snake = df_dropped.toDF(*new_columns)
df_snake.columns

['age',
 'attrition',
 'business_travel',
 'daily_rate',
 'department',
 'distance_from_home',
 'education',
 'education_field',
 'employee_number',
 'environment_satisfaction',
 'gender',
 'hourly_rate',
 'job_involvement',
 'job_level',
 'job_role',
 'job_satisfaction',
 'marital_status',
 'monthly_income',
 'monthly_rate',
 'num_companies_worked',
 'over_time',
 'percent_salary_hike',
 'performance_rating',
 'relationship_satisfaction',
 'stock_option_level',
 'total_working_years',
 'training_times_last_year',
 'work_life_balance',
 'years_at_company',
 'years_in_current_role',
 'years_since_last_promotion',
 'years_with_curr_manager']

In [6]:
df_snake.dropDuplicates().count()
#This data set has no duplicate rows

1470

In [7]:
df_snake.select([count(when(isnan(i), i)).alias(i) for i in df_snake.columns]).show()
#This dataset has no NaN values. I have also confirmed this on data wrangler in VSCode 

+---+---------+---------------+----------+----------+------------------+---------+---------------+---------------+------------------------+------+-----------+---------------+---------+--------+----------------+--------------+--------------+------------+--------------------+---------+-------------------+------------------+-------------------------+------------------+-------------------+------------------------+-----------------+----------------+---------------------+--------------------------+-----------------------+
|age|attrition|business_travel|daily_rate|department|distance_from_home|education|education_field|employee_number|environment_satisfaction|gender|hourly_rate|job_involvement|job_level|job_role|job_satisfaction|marital_status|monthly_income|monthly_rate|num_companies_worked|over_time|percent_salary_hike|performance_rating|relationship_satisfaction|stock_option_level|total_working_years|training_times_last_year|work_life_balance|years_at_company|years_in_current_role|years_si

In [8]:
url = "jdbc:postgresql://localhost:5432/HR"
properties = {
    "user": "postgres",
    "password": "Cheesecake",
    "driver": "org.postgresql.Driver"
}

df_snake.write.jdbc(url=url, table="HR_Employee_Attrition", properties=properties, mode="overwrite")

In [9]:
import pandas as pd

pandas_df = df_snake.toPandas()

pandas_df
pandas_df.to_csv("updated_HR_employee_attrition.csv")

In [10]:
df_snake.createOrReplaceTempView("table_view")
# df_queried = spark.sql("SELECT * FROM table_view")
# df_queried.show()

In [11]:
avg_age_query = """
    SELECT AVG(age) AS average_age
    FROM table_view
"""
spark.sql(avg_age_query).show()

+------------------+
|       average_age|
+------------------+
|36.923809523809524|
+------------------+



In [12]:
most_pop_department = """
    SELECT department, COUNT(*) as count
    FROM table_view
    GROUP BY department
    ORDER BY count DESC
    LIMIT 1;
"""
spark.sql(most_pop_department).show()

+--------------------+-----+
|          department|count|
+--------------------+-----+
|Research & Develo...|  961|
+--------------------+-----+



In [15]:
median_dist_from_home = """
    SELECT MEDIAN(distance_from_home)
    FROM table_view
"""
spark.sql(median_dist_from_home).show()

+--------------------------+
|median(distance_from_home)|
+--------------------------+
|                       7.0|
+--------------------------+



In [14]:
most_common_lvl_of_edu = """
    SELECT education, COUNT(*) as count
    FROM table_view
    GROUP BY education
    ORDER BY COUNT DESC
    LIMIT 1;
"""
spark.sql(most_common_lvl_of_edu).show()

+---------+-----+
|education|count|
+---------+-----+
|        3|  572|
+---------+-----+

